In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset="train",shuffle=True)
newsgroups_test = fetch_20newsgroups(subset="test",shuffle=True)

In [2]:
print(list(newsgroups_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


## Now Data preprocessing to feed to LDA model
The data will be be processed in the following steps
- Tokenization : Split the text into sentence and sentences into words,also all words are lowered and punctuation is removed
- All stopwords are removed, i.e helping verbs(mostly)
- Words are lemmatized : words in third person are changed to first person and words in past and future tenses are changed to present
- Words are stemmed : words are converted to there base class like dancing is converted to danc which is root of the word

In [5]:
!pip install gensim

In [7]:
import numpy as np 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *

In [8]:
np.random.seed(42)

In [14]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/pranavprajapati/nltk_data...


True

### Lemmatizer example

In [15]:
print(WordNetLemmatizer().lemmatize('ran',pos='v'))

run


### Stemmer example

In [18]:
import pandas as pd
stemmer = SnowballStemmer('english')
original_words = ["dancing", "loving" , "horses","ran", "sensational", "beautifully", "flies", "dies"]
base_words = [stemmer.stem(words) for words in original_words]
pd.DataFrame(data={"orginal_words":original_words,"base_words":base_words})

,orginal_words,base_words
0,dancing,danc
1,loving,love
2,horses,hors
3,ran,ran
4,sensational,sensat
5,beautifully,beauti
6,flies,fli
7,dies,die


In [19]:
"""
    Function to preprocess data in the entire dataset
"""
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos=v))

def preprocessor(text):
    result =[]
    for token in gensim.utils.simple_preprocess():
        if token is not gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
    return result